In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('run-data.xlsx')

In [3]:
data.shape

(14, 31)

In [4]:
# data.drop(len(data)-1, inplace=True)

In [5]:
# data.to_excel('../run-data.xlsx', index=False)

In [6]:
data.head()

,rollout,waitingTime,time,rew,Ep_num_train_rollouts,Ep_rollout_length,Ep_eval_freq,Ep_eval_num_eps,Ep_max_ep_steps,Ep_generation_ep_steps,...,P_gradient_clip,P_value_loss_coef,R_rule_set,R_rule_set_params,En_shape,En_rush_hour,En_uniform_generation_probability,M_reward_interpolation,M_state_interpolation,P_num_workers
0,150,545.087500,56.127909,1131.958000,400,2048,50,25,1250,1000,...,1,1,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0.0,0,8
1,407,497.771667,55.738751,1143.073556,400,2048,50,25,1250,1000,...,1,1,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0.0,0,8
2,408,527.901667,57.210245,1135.121111,400,2048,50,25,1250,1000,...,1,1,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0.0,0,8
3,408,503.400833,56.050249,1140.997444,400,2048,50,25,1250,1000,...,1,1,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0.2,0,8
4,350,514.990833,56.638730,1137.633556,400,2048,50,25,1250,1000,...,1,1,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0.2,0,8


In [32]:
data[data['waitingTime'] < 1000]`

,rollout,waitingTime,time,rew,E_num_train_rollouts,E_rollout_length,E_eval_freq,E_eval_num_eps,E_max_ep_steps,E_test_num_eps,...,A_optimization_epochs,A_ppo_ratio_clip,A_discount,A_learning_rate,A_clip_grads,A_gradient_clip,A_value_loss_coef,O_num_agents,O_rule_set,O_rule_set_params
30,258,994.88,4.290420,645.457778,250,1024,98,25,1000,250,...,5,0.3,0.99,0.00001,True,1,1.0,8,NaN,{'length': 5}
118,251,998.33,5.760454,641.853333,250,1024,98,25,1000,250,...,15,0.3,0.99,0.00001,False,1,1.0,8,NaN,{'length': 5}


# Notes
- rollout_length: at least 1024 (2048 is better)
- gae_tau: at most 0.9 (maybe lower)
- entropy_weight: at least 0.1 (test higher)
- minibatch_size: at least 128 (maybe higher)
- optimization_epochs: 5 but 10 is very close
- ppo_ratio_clip: 0.3 but 0.2 is close
- clip_grads: true
- value_loss_coef: 1.0
- reward_interpolation: 0 


- NS_mult: 1.0 or 1.5
- phase_end_offset: any (0 - 100)
- multi intersection cycle length: 20
- (best combo 1.5, 100, 20)

In [8]:
data[data['M_reward_interpolation'] == 0]['waitingTime'].mean()

523.5869444444444

In [9]:
data[data['M_reward_interpolation'] == 0.2]['waitingTime'].mean()

510.1986111111111

In [10]:
data[data['M_reward_interpolation'] == 0.5]['waitingTime'].mean()

567.0277777777777

In [11]:
data[data['M_reward_interpolation'] == 0.8]['waitingTime'].mean()

615.3183333333333

In [12]:
data[data['M_reward_interpolation'] == 1]['waitingTime'].mean()

600.2083333333333